<a href="https://colab.research.google.com/github/Ruthwik24/Deep-Face-Recognition-Using-an-ZFNet-Architecture/blob/main/Deep_Face_Recognition_Using_an_ZFNet_Architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from PIL import Image

# Source folder where the images are currently located
source_folder = r'/content/drive/MyDrive/archive'

# Target folder where we will organize the images
processed_data_folder = r'/content/drive/MyDrive/Preprocessed'

# Check if the source folder exists
if not os.path.exists(source_folder):
    print(f"Source folder '{source_folder}' does not exist.")
else:
    # Create the processed folder if it doesn't exist
    if not os.path.exists(processed_data_folder):
        os.makedirs(processed_data_folder)

    # Process each file in the source folder
    for filename in os.listdir(source_folder):
        # Ignore non-image files (e.g., Readme)
        if not filename.startswith("subject") or len(filename.split('.')) < 2:
            continue

        # Extract subject and expression from the filename
        parts = filename.split('.')
        subject = parts[0]  # e.g., "subject01"
        expression = parts[1]  # e.g., "centerlight"

        # Create a folder for each subject if it doesn't exist
        subject_folder = os.path.join(processed_data_folder, subject)
        if not os.path.exists(subject_folder):
            os.makedirs(subject_folder)

        # Define the destination path for the converted image
        dst_path = os.path.join(subject_folder, f"{expression}.jpg")

        # Convert and save the image to JPEG format
        src_path = os.path.join(source_folder, filename)
        try:
            with Image.open(src_path) as img:
                img = img.convert('RGB')  # Convert to RGB format if needed
                img.save(dst_path, 'JPEG')
                print(f"Converted and saved: {dst_path}")  # Print each conversion for confirmation
        except Exception as e:
            print(f"Could not process file {filename}: {e}")

Converted and saved: /content/drive/MyDrive/Preprocessed/subject02/normal.jpg
Converted and saved: /content/drive/MyDrive/Preprocessed/subject01/glasses.jpg
Converted and saved: /content/drive/MyDrive/Preprocessed/subject01/normal.jpg
Converted and saved: /content/drive/MyDrive/Preprocessed/subject01/rightlight.jpg
Converted and saved: /content/drive/MyDrive/Preprocessed/subject01/sleepy.jpg
Converted and saved: /content/drive/MyDrive/Preprocessed/subject01/leftlight.jpg
Converted and saved: /content/drive/MyDrive/Preprocessed/subject01/wink.jpg
Converted and saved: /content/drive/MyDrive/Preprocessed/subject02/glasses.jpg
Converted and saved: /content/drive/MyDrive/Preprocessed/subject01/noglasses.jpg
Converted and saved: /content/drive/MyDrive/Preprocessed/subject01/sad.jpg
Converted and saved: /content/drive/MyDrive/Preprocessed/subject02/noglasses.jpg
Converted and saved: /content/drive/MyDrive/Preprocessed/subject02/happy.jpg
Converted and saved: /content/drive/MyDrive/Preprocesse

In [ ]:
import os
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# ZFNet-Inspired Basic Model
def build_basic_zfnet_model(input_shape, num_classes):
    model = models.Sequential()

    # First Convolutional Block
    model.add(layers.Conv2D(48, (7, 7), strides=(2, 2), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    # Second Convolutional Block
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Third Convolutional Block
    model.add(layers.Conv2D(192, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Fully Connected Layers
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))  # Simplified dense layer size
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

# Set up data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

input_shape = (224, 224, 3)
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Preprocessed',
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Preprocessed',
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='validation'
)

# Initialize and compile the ZFNet-inspired model
num_classes = train_generator.num_classes
model = build_basic_zfnet_model(input_shape, num_classes)
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Set up callbacks
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=45,
    callbacks=[lr_scheduler, early_stopping]
)

# Evaluate the model
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Validation accuracy: {val_accuracy * 100:.2f}%")

Found 135 images belonging to 15 classes.
Found 30 images belonging to 15 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/45


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 876ms/step - accuracy: 0.0872 - loss: 2.7172 - val_accuracy: 0.0667 - val_loss: 2.6852 - learning_rate: 1.0000e-04
Epoch 2/45
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 544ms/step - accuracy: 0.1245 - loss: 2.6764 - val_accuracy: 0.1667 - val_loss: 2.6649 - learning_rate: 1.0000e-04
Epoch 3/45
9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 837ms/step - accuracy: 0.2108 - loss: 2.6310 - val_accuracy: 0.2333 - val_loss: 2.6240 - learning_rate: 1.0000e-04
Epoch 4/45
9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 537ms/step - accuracy: 0.1910 - loss: 2.5973 - val_accuracy: 0.3333 - val_loss: 2.5756 - learning_rate: 1.0000e-04
Epoch 5/45
9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 846ms/step - accuracy: 0.2256 - loss: 2.5556 - val_accuracy: 0.3333 - val_loss: 2.5078 - learning_rate: 1.0000e-04
Epoch 6/45
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 546ms/step - accuracy: 0.2898 - loss: 2.4672 - val_accuracy: 0.3333 - val_loss: 2.4093 - learning_rate: 1.0000e-04
Epoch 7/45
9/9 ━━━━━━━━━━━━━━━━━━━━ 12s 694ms/step - accuracy: 0.2192 - loss: 2.4379 -